<h1> Sophos AV - Compile Installer Strings </h1>

# Import Modules and Env

In [ ]:
# data import and file manipulation
import os
import json
import csv
import xlrd
import zipfile

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# source dir for nable exported data
source_folder = 'd:/project_docs/abc_nable_migration/abc_nable_exports/patch_management'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

### Import Modules, Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Sophos DataFrame Creation

In [ ]:
import requests
from requests.structures import CaseInsensitiveDict

sophos_config = config['sophoscentral']

base_uri = sophos_config['base_uri']

In [ ]:
# call token api url
token_uri = 'https://id.sophos.com/api/v2/oauth2/token'

# construct header
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/x-www-form-urlencoded"

# construct req body
data = f"grant_type=client_credentials&client_id={sophos_config['client_id']}&client_secret={sophos_config['client_secret']}&scope=token"

# request content response
resp = requests.post(token_uri, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# create auth and refresh tokens
access_token = c_dict['access_token']
refresh_token = c_dict['refresh_token']

## Partner ID Lookup

In [ ]:
# Partner ID lookup
whoami = 'https://api.central.sophos.com/whoami/v1'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'

# request partner id
resp = requests.get(whoami, headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# store partner id
partner_id = c_dict['id']

## Pull Tenant Info

### Filter Data for Active

In [ ]:
def active_site(site):
    if site['billingType'] == 'trial':
        return None
    elif site['dataRegion'] == None:
        return None
    elif re.match(r'inactive',str(site['showAs']).lower()):
        return None
    elif site['status'] == 'active':
        return site

In [ ]:
# Tenant ID lookup
tenants_url = 'https://api.central.sophos.com/partner/v1/tenants'

# construct header
headers = CaseInsensitiveDict()

headers['Authorization'] = f'Bearer {access_token}'
headers['X-Partner-ID'] = f'{partner_id}'
headers['Accept'] = 'application/json'
headers['pageSize'] = '100'


resp = requests.get(tenants_url,headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


sophos_tenants = []

for site in c_dict['items']:
    filtered_site = active_site(site)
    if filtered_site:
        sophos_tenants.append(site)

In [ ]:
for tenant in sophos_tenants:
    print(f'partnerID: ',partner_id)
    print(tenant['showAs'])
    print(tenant['id'])
    print(tenant['apiHost'])

## Pull Tenant Info

In [ ]:
dataRegion = "us03"

for tenant in sophos_tenants:

    # Tenant Download URL lookup
    tenant_download_url = f"{tenant['apiHost']}/endpoint/v1/downloads?requestedProducts=coreAgent&platforms=windows"
    print(tenant_download_url)

    # construct header
    headers = CaseInsensitiveDict()

    headers['Authorization'] = f"Bearer {access_token}"
    headers['X-Partner-ID'] = f'{partner_id}'
    headers['X-Tenant-ID'] = f"{tenant['id']}"
    headers['Accept'] = 'application/json'
    # headers['X-Tenant-ID'] = f"{partner_id}"
    # headers['pageSize'] = '100'

    resp = requests.get(tenant_download_url,headers=headers)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict)
    print(resp.content)

In [ ]:
resp.content

In [ ]:
base_uri

In [ ]:
sophos_tenants

# Sophos Install String Templates

In [ ]:
winos_install_template = f"""

SophosSetup.exe --customertoken={client['siteId']} --epinstallerserver="api-cloudstation-us-east-2.prod.hydra.sophos.com" --products="antivirus,intercept" --quiet

"""

# Create Datto DataFrame

In [ ]:
# import and assign secrets from env.ini

dattormm_config = config['dattormm']

base_uri = dattormm_config['base_uri']

## Create auth token

In [ ]:
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/api/v2/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)